# Hands-on 2: How to create a fMRI analysis workflow

The purpose of this section is that you setup a fMRI analysis workflow. 


# 1st-level Analysis Workflow Structure

In this notebook we will create a workflow that performs 1st-level analysis and normalizes the resulting beta weights to the MNI template. In concrete steps this means:

    1. Specify 1st-level model parameters
    2. Specify 1st-level contrasts
    3. Estimate 1st-level contrasts
    4. Normalize 1st-level contrasts

## Imports

It's always best to have all relevant module imports at the beginning of your script. So let's import what we most certainly need.

In [ ]:
from nilearn import plotting
%matplotlib inline

# Get the Node and Workflow object
from nipype import Node, Workflow

# Specify which SPM to use
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-dev/spm12_mcr/spm/spm12')

## Create Nodes and Workflow connections

Let's create all the nodes that we need! Make sure to specify all relevant inputs and keep in mind which ones you later on need to connect in your pipeline.


### Specify 1st-level model parameters (stimuli onsets, duration, etc.)

The specify the 1st-level model we need the subject specific onset times and durations of the stimuli. Luckily, as we are working with a BIDS dataset, this information is nicely stored in a `tsv` file:

In [ ]:
import pandas as pd

# Create the workflow here
analysis1st = Workflow(name='work_1st', base_dir='/output/')
trialinfo = pd.read_table('/data/ds000114/task-fingerfootlips_events.tsv')
trialinfo


In [ ]:
import pandas as pd
from nipype.interfaces.base import Bunch
for group in trialinfo.groupby('trial_type'):
    print(group)
    print("")
trialinfo = pd.read_table('/data/ds000114/task-fingerfootlips_events.tsv')
conditions = []
onsets = []
durations = []

for group in trialinfo.groupby('trial_type'):
    conditions.append(group[0])
    onsets.append(list(group[1].onset -10)) # subtracting 10s due to removing of 4 dummy scans
    durations.append(group[1].duration.tolist())

subject_info = [Bunch(conditions=conditions,
                      onsets=onsets,
                      durations=durations,
                      )]

from nipype.algorithms.modelgen import SpecifySPMModel

# Initiate the SpecifySPMModel node here
modelspec = Node(SpecifySPMModel(concatenate_runs=False,
                                 input_units='secs',
                                 output_units='secs',
                                 time_repetition=2.5,
                                 high_pass_filter_cutoff=128,
                                 subject_info=subject_info),
                 name="modelspec")

This node will also need some additional inputs, such as the preprocessed functional images, the motion parameters etc. We will specify those once we take care of the workflow data input stream.

### Specify 1st-level contrasts

To do any GLM analysis, we need to also define the contrasts that we want to investigate. If we recap, we had three different conditions in the **fingerfootlips** task in this dataset:

- **finger**
- **foot**
- **lips**

Therefore, we could create the following contrasts (seven T-contrasts and two F-contrasts):

In [ ]:
# Condition names
condition_names = ['Finger', 'Foot', 'Lips']

# Contrasts
cont01 = ['average',        'T', condition_names, [1/3., 1/3., 1/3.]]
cont02 = ['Finger',         'T', condition_names, [1, 0, 0]]
cont03 = ['Foot',           'T', condition_names, [0, 1, 0]]
cont04 = ['Lips',           'T', condition_names, [0, 0, 1]]
cont05 = ['Finger > others','T', condition_names, [1, -0.5, -0.5]]
cont06 = ['Foot > others',  'T', condition_names, [-0.5, 1, -0.5]]
cont07 = ['Lips > others',  'T', condition_names, [-0.5, -0.5, 1]]

cont08 = ['activation',     'F', [cont02, cont03, cont04]]
cont09 = ['differences',    'F', [cont05, cont06, cont07]]

contrast_list = [cont01, cont02, cont03, cont04, cont05, cont06, cont07, cont08, cont09]

### Estimate 1st-level contrasts

Before we can estimate the 1st-level contrasts, we first need to create the 1st-level design. Here you can also specify what kind of basis function you want (HRF, FIR, Fourier, etc.), if you want to use time and dispersion derivatives and how you want to model the serial correlation.

In this example I propose that you use an HRF basis function, that we model time derivatives and that we model the serial correlation with AR(1).

In [ ]:
from nipype.interfaces.spm import Level1Design
# Initiate the Level1Design node here
level1design = Node(Level1Design(bases={'hrf': {'derivs': [1, 0]}},
                                 timing_units='secs',
                                 interscan_interval=2.5,
                                 model_serial_correlations='AR(1)'),
                    name="level1design")


# Now that we have the Model Specification and 1st-Level Design node, we can connect them to each other:
# Connect the two nodes here
analysis1st.connect([(modelspec, level1design, [('session_info',
                                                 'session_info')])])

# Now we need to estimate the model. I recommend that you'll use a Classical: 1 method to estimate the model.
from nipype.interfaces.spm import EstimateModel
# Initiate the EstimateModel node here
level1estimate = Node(EstimateModel(estimation_method={'Classical': 1}),
                      name="level1estimate")

# Now we can connect the 1st-Level Design node with the model estimation node.
# Connect the two nodes here
analysis1st.connect([(level1design, level1estimate, [('spm_mat_file',
                                                      'spm_mat_file')])])
from nipype.interfaces.spm import EstimateContrast
# Initiate the EstimateContrast node here
level1conest = Node(EstimateContrast(contrasts=contrast_list),
                    name="level1conest")

# Now we can connect the model estimation node with the contrast estimation node.
analysis1st.connect([(level1estimate, level1conest, [('spm_mat_file',
                                                      'spm_mat_file'),
                                                     ('beta_images',
                                                      'beta_images'),
                                                     ('residual_image',
                                                      'residual_image')])])

## Normalize 1st-level contrasts

Now that the contrasts were estimated in subject space we can put them into a common reference space by normalizing them to a specific template. In this case we will be using SPM12's Normalize routine and normalize to the SPM12 tissue probability map `TPM.nii`.

At this step you can also specify the voxel resolution of the output volumes. If you don't specify it, it will normalize to a voxel resolution of 2x2x2mm. 

In [ ]:
from nipype.interfaces.spm import Normalize12

# Location of the template
template = '/opt/spm12-dev/spm12_mcr/spm/spm12/tpm/TPM.nii'

# Initiate the Normalize12 node here
normalize = Node(Normalize12(jobtype='estwrite',
                             tpm=template,
                             write_voxel_sizes=[2, 2, 2]
                            ),
                 name="normalize")

# Connect the nodes here
analysis1st.connect([(level1conest, normalize, [('spmT_images',
                                                 'apply_to_files')])
                     ])

## Datainput with `SelectFiles` and `iterables` 

In [ ]:
# Import the SelectFiles
from nipype import SelectFiles

# String template with {}-based strings
templates = {'anat': '/data/ds000114/sub-{subj_id}/ses-test/anat/sub-{subj_id}_ses-test_T1w.nii.gz',
             'func': '/output/datasink_handson/preproc/sub-{subj_id}_detrend.nii.gz',
             'mc_param': '/output/datasink_handson/preproc/sub-{subj_id}.par',
             'outliers': '/output/datasink_handson/preproc/art.sub-{subj_id}_outliers.txt'
            }

# Create SelectFiles node
sf = Node(SelectFiles(templates, sort_filelist=True),
          name='selectfiles')

# Now we can specify over which subjects the workflow should iterate. 
# list of subject identifiers
subject_list = ['07']
sf.iterables = [('subj_id', subject_list)]


# Gunzip Node

from nipype.algorithms.misc import Gunzip
# Initiate the two Gunzip node here
gunzip_anat = Node(Gunzip(), name='gunzip_anat')
gunzip_func = Node(Gunzip(), name='gunzip_func')


# And as a final step, we just need to connect this SelectFiles node to the rest of the workflow.
# Connect SelectFiles node to the other nodes here
analysis1st.connect([(sf, gunzip_anat, [('anat', 'in_file')]),
                     (sf, gunzip_func, [('func', 'in_file')]),
                     (gunzip_anat, normalize, [('out_file', 'image_to_align')]),
                     (gunzip_func, modelspec, [('out_file', 'functional_runs')]),
                     (sf, modelspec, [('mc_param', 'realignment_parameters'),
                                      ('outliers', 'outlier_files'),
                                      ])
                    ])

#Data output with DataSink
#Now, before we run the workflow, let's again specify a Datasink folder to only keep those files that we want to keep.
from nipype.interfaces.io import DataSink
# Initiate DataSink node here
# Initiate the datasink node
output_folder = 'datasink_handson'
datasink = Node(DataSink(base_directory='/output/',
                         container=output_folder),
                name="datasink")
## Use the following substitutions for the DataSink output
substitutions = [('_subj_id_', 'sub-')]
datasink.inputs.substitutions = substitutions

# Connect nodes to datasink here
analysis1st.connect([(level1conest, datasink, [('spm_mat_file', '1stLevel.@spm_mat'),
                                               ('spmT_images', '1stLevel.@T'),
                                               ('spmF_images', '1stLevel.@F'),
                                              ]),
                     (normalize, datasink, [('normalized_files', 'normalized.@files'),
                                            ('normalized_image', 'normalized.@image'),
                                           ]),
                    ])

## Visualize the workflow

Now that the workflow is finished, let's visualize it again.

In [ ]:
# Create 1st-level analysis output graph
analysis1st.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename='/output/work_1st/graph.png')

##  Run the Workflow

Now that everything is ready, we can run the 1st-level analysis workflow. Change ``n_procs`` to the number of jobs/cores you want to use.

In [ ]:
analysis1st.run('MultiProc', plugin_args={'n_procs': 4})

## Visualize results

### First, let's look at the 1st-level Design Matrix of subject one, to verify that everything is as it should be.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import loadmat

# Using scipy's loadmat function we can access SPM.mat
spmmat = loadmat('/output/datasink_handson/1stLevel/sub-07/SPM.mat',
                 struct_as_record=False)
designMatrix = spmmat['SPM'][0][0].xX[0][0].X
names = [i[0] for i in spmmat['SPM'][0][0].xX[0][0].name[0]]
normed_design = designMatrix / np.abs(designMatrix).max(axis=0)
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(normed_design, aspect='auto', cmap='gray', interpolation='none')
ax.set_ylabel('Volume id')
ax.set_xticks(np.arange(len(names)))
ax.set_xticklabels(names, rotation=90);

### Let's look how well the normalization worked.

In [ ]:
import nibabel as nb
from nilearn.plotting import plot_anat
from nilearn.plotting import plot_glass_brain
# Load GM probability map of TPM.nii
img = nb.load('/opt/spm12-dev/spm12_mcr/spm/spm12/tpm/TPM.nii')
GM_template = nb.Nifti1Image(img.get_data()[..., 0], img.affine, img.header)

# Plot normalized subject anatomy
display = plot_anat('/output/datasink_handson/normalized/sub-07/wsub-07_ses-test_T1w.nii',
                    dim=-0.1)

# Overlay in edges GM map
display.add_edges(GM_template)

# Plot raw subject anatomy
display = plot_anat('/data/ds000114/sub-07/ses-test/anat/sub-07_ses-test_T1w.nii.gz',
                    dim=-0.1)

# Overlay in edges GM map
display.add_edges(GM_template)

### Let's look at the contrasts of one subject that we've just computed.

In [ ]:
from nilearn.plotting import plot_stat_map
anatimg = '/data/ds000114/sub-07/ses-test/anat/sub-07_ses-test_T1w.nii.gz'
plot_stat_map('/output/datasink_handson/1stLevel/sub-07/spmT_0001.nii', title='average',
    bg_img=anatimg, threshold=3, display_mode='y', cut_coords=(-5, 0, 5, 10, 15), dim=-1);
plot_stat_map('/output/datasink_handson/1stLevel/sub-07/spmT_0002.nii', title='finger',
    bg_img=anatimg, threshold=3, display_mode='y', cut_coords=(-5, 0, 5, 10, 15), dim=-1);
plot_stat_map('/output/datasink_handson/1stLevel/sub-07/spmT_0003.nii', title='foot',
    bg_img=anatimg, threshold=3, display_mode='y', cut_coords=(-5, 0, 5, 10, 15), dim=-1);
plot_stat_map('/output/datasink_handson/1stLevel/sub-07/spmT_0004.nii', title='lip',
    bg_img=anatimg, threshold=3, display_mode='y', cut_coords=(-5, 0, 5, 10, 15), dim=-1);

### We can also check three additional contrasts Finger > others, Foot > others and Lips > others.

In [ ]:
plot_stat_map('/output/datasink_handson/1stLevel/sub-07/spmT_0005.nii', title='fingers > other',
    bg_img=anatimg, threshold=3, display_mode='y', cut_coords=(-5, 0, 5, 10, 15), dim=-1);
plot_stat_map('/output/datasink_handson/1stLevel/sub-07/spmT_0006.nii', title='foot > other',
    bg_img=anatimg, threshold=3, display_mode='y', cut_coords=(-5, 0, 5, 10, 15), dim=-1);
plot_stat_map('/output/datasink_handson/1stLevel/sub-07/spmT_0007.nii', title='lip > other',
    bg_img=anatimg, threshold=3, display_mode='y', cut_coords=(-5, 0, 5, 10, 15), dim=-1);

### We can plot the normalized results over a template brain

In [ ]:
plot_glass_brain('/output/datasink_handson/normalized/sub-07/wspmT_0005.nii',
                 colorbar=True, display_mode='lyrz', black_bg=True, threshold=3,
                 title='fingers>other');
plot_glass_brain('/output/datasink_handson/normalized/sub-07/wspmT_0006.nii',
                 colorbar=True, display_mode='lyrz', black_bg=True, threshold=3,
                 title='foot>other');
plot_glass_brain('/output/datasink_handson/normalized/sub-07/wspmT_0007.nii',
                 colorbar=True, display_mode='lyrz', black_bg=True, threshold=3,
                 title='lip>other');